In [ ]:
!pip install -r requirements.txt

In [6]:
from elasticsearch import Elasticsearch
from getpass import getpass
import pprint

pp = pprint.PrettyPrinter(indent=2)

In [3]:
cloud_id = getpass("Elastic deployment Cloud ID: ")
cloud_api_key = getpass("Elastic deployment API Key: ")
INDEX_NAME = 'books'
MODEL_ID = 'sentence-transformers__msmarco-minilm-l-12-v3'

es = Elasticsearch(
    cloud_id=cloud_id,
    api_key=cloud_api_key,
)

query_string = "Dinosaurs are still alive"

Elastic deployment Cloud ID:  ········
Elastic deployment API Key:  ········


In [29]:
def print_results(search_result):
    print(f"Total hits: {search_result['hits']['total']['value']}")
    for hit in search_result["hits"]["hits"]:
        print(f"Book: {hit['_source']['book_title']}")
        print(f"Author: {hit['_source']['author_name']}")
        print(f"Description: {hit['_source']['book_description']}")
        print(f"Score: {hit['_score']}")
        print("")

### 1. Vector Search Example

In [30]:
vector_search_result = es.search(
        index = INDEX_NAME,
        knn={
            "field": "description_embedding",
            "k": 10,
            "num_candidates": 50,
            "query_vector_builder": {
                "text_embedding": {"model_id": MODEL_ID, "model_text": query_string}
            },
        },
    )

print_results(vector_search_result)

Total hits: 10
Book: Mage Against the Machine
Author: Shaun Barger
Description: Harry Potter meets The Terminator in this action-packed adventure about a young man who discovers that everything he believed about his world is a lie.The year is 2120. The humans are dead. The mages have retreated from the world after a madman blew up civilization with weaponized magical technology. Safe within domes that protect them from the nuclear wasteland on the other side, the mages have spent the last century putting their lives back together. Nikolai is obsessed with artifacts from twentieth-century human life: mage-crafted replica Chuck Taylors on his feet, Schwarzenegger posters on his walls, Beatlemania still alive and well in his head. But he’s also tasked with a higher calling—to maintain the Veils that protect mage-kind from the hazards of the wastes beyond. As a cadet in the Mage King’s army, Nik has finally found what he always wanted—a purpose. But when confronted by one of his former ins

### 2. Full-Text Search Example

In [37]:
bm25_result = es.search(
        index=INDEX_NAME, 
        body={
            "query": {
                "match": {
                    "book_description": query_string
                }
            },
            "size": 10
        },
        
    )

print_results(bm25_result)

Total hits: 5193
Book: Unnatural History
Author: Jonathan Green
Description: In the closing years of the 20th century the British Empire's rule is still going strong. Queen Victoria is about to celebrate her 160th birthday, kept alive by advanced steam technology. London is a fantastical sprawling metropolis where dirigibles roam the skies, robot bobbies enforce the law and dinosaurs are on display in London zoo. Welcome to Magna Britannia, a steam driven world full of fantastical creations and shady villains. Here dashing dandies and mustachioed villains battle for supremacy while below the city strange things stir in the flooded tunnels of the old London Underground. In two scant months the nation, and all her colonies, will celebrate 160 years of Queen Victoria's glorious reign. But all is not well at the heart of the empire of Magna Britannia. A chain of events is about to be set in motion that, if not stopped, could lead to a world-shattering conclusion. It begins with a break-in 

### 3. Hybrid Search Example

In [38]:
hybrid_search_result = es.search(
    index=INDEX_NAME,
    body={
        "query": {
            "bool": {
                "must": [
                    {"match": 
                        {
                            "book_description": query_string,
                        }
                    },
                ]
            }
        },
        "knn": {
            "field": "description_embedding",
            "k": 10,
            "query_vector_builder": {
                "text_embedding": {"model_id": MODEL_ID, "model_text": query_string}
            },
        },
        "rank": {
            "rrf": {}
        },
        "size": 10
    },
)

print_results(hybrid_search_result)

Total hits: 5195
Book: West of Eden
Author: Harry Harrison
Description: Sixty-five million years ago, a disastrous cataclysm eliminated three quarters of all life on Earth. Overnight, the age of dinosaurs ended. The age of mammals had begun.But what if history had happened differently? What if the reptiles had survived to evolve intelligent life?In West of Eden, bestselling author Harry Harrison has created a rich, dramatic saga of a world where the descendents of the dinosaurs struggled with a clan of humans in a battle for survival.Here is the story of Kerrick, a young hunter who grows to manhood among the dinosaurs, escaping at last to rejoin his own kind. His knowledge of their strange customs makes him the humans' leader...and the dinosaurs' greatest enemy.Rivalling Frank Herbert's Dune in the majesty of its scope and conception, West of Eden is a monumental epic of love and savagery, bravery and hope.
Score: 0.03053613

Book: Unnatural History
Author: Jonathan Green
Description: 